In [1]:
import matplotlib.pyplot as plt


def plot_all_metric_and_model_comparation():
    models_metrics = {}

    titles = [
        "ROC_AUC_Scores_Comparison",
        "F1_Scores_Comparison",
        "Accuracy_Comparison"
    ]

    with open("data_generated/test/test_all_summary.txt", "r") as file:
        lines = file.readlines()
        for line in lines:
            if line.startswith("Type"):  # Skip header or unrelated lines
                continue
            parts = line.split(": ", 1)
            if len(parts) == 2:
                model_name = parts[0].strip()
                metrics = eval(parts[1].strip())  # Convert string representation of dictionary to actual dictionary
                models_metrics[model_name] = metrics

    roc_auc_scores = {model: metrics["ROC AUC"] for model, metrics in models_metrics.items()}
    f1_scores = {model: metrics["F1 Score"] for model, metrics in models_metrics.items()}
    accuracies = {model: metrics["Accuracy"] for model, metrics in models_metrics.items()}
    colors = ["blue", "green", "red", "purple", "orange"]

    for metrics, title in zip([roc_auc_scores, f1_scores, accuracies], titles):
        names = list(metrics.keys())
        values = list(metrics.values())
        plt.figure(figsize=(10, 6))  # You can adjust the figure size
        plt.bar(names, values, color=colors[:len(metrics)])
        plt.ylabel("Score")
        plt.title(title.replace('_', ' '))
        plt.xticks(rotation=45)
        plt.savefig(f"data_generated/test/plots/{title}.png")
        plt.close()
plot_all_metric_and_model_comparation()

